# Pruebas MultiFADL One-versus-Rest. Brain dataset

In [12]:
import pickle
import json

import pandas as pd
import tensorflow as tf
import numpy as np
from src import utils

from src.MonoFADLModel import MonoFADLModel
from src.MultiFADLModelOvR import MultiFADLModelOvR
from src.NoSelectionModel import NoSelectionModel

# Seed for neural network executions
SEED = 1234
np.random.seed(SEED)
tf.random.set_seed(SEED)


In [13]:
comparative_results = pd.DataFrame(columns=['test_loss', 'test_accuracy', 'test_f1', 'number of selected features', 'selected Features'])

* Preprocesamiento

In [14]:
# Load dataset and preprocess it
# https://www.kaggle.com/datasets/brunogrisci/brain-cancer-gene-expression-cumida

# Check if stored, otherwise load and store
try:
    brain = pickle.load(open('data/brain_GSE50161.pkl', 'rb'))
except:
    brain = pd.read_csv('data/brain_GSE50161.csv')
    brain = brain.drop(columns=['samples'])

    brain['type'] = brain['type'].map({
        'normal': 0,
        'ependymoma': 1,
        'glioblastoma': 2,
        'medulloblastoma': 3,
        'pilocytic_astrocytoma': 4
    })

    with open('data/brain_GSE50161.pkl', 'wb') as f:
        pickle.dump(brain, f)
        
brain

,type,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,1405_i_at,...,AFFX-r2-Ec-bioD-3_at,AFFX-r2-Ec-bioD-5_at,AFFX-r2-P1-cre-3_at,AFFX-r2-P1-cre-5_at,AFFX-ThrX-3_at,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at
0,1,12.498150,7.604868,6.880934,9.027128,4.176175,7.224920,6.085942,6.835999,5.898355,...,9.979005,9.926470,12.719785,12.777792,5.403657,4.870548,4.047380,3.721936,4.516434,4.749940
1,1,13.067436,7.998090,7.209076,9.723322,4.826126,7.539381,6.250962,8.012549,5.453147,...,11.924749,11.215930,13.605662,13.401342,5.224555,4.895315,3.786437,3.564481,4.430891,4.491416
2,1,13.068179,8.573674,8.647684,9.613002,4.396581,7.813101,6.007746,7.178156,8.400266,...,12.154405,11.532460,13.764593,13.477800,5.303565,5.052184,4.005343,3.595382,4.563494,4.668827
3,1,12.456040,9.098977,6.628784,8.517677,4.154847,8.361843,6.596064,6.347285,4.900380,...,11.969072,11.288801,13.600828,13.379029,4.953429,4.708371,3.892318,3.759429,4.748381,4.521275
4,1,12.699958,8.800721,11.556188,9.166309,4.165891,7.923826,6.212754,6.866387,5.405628,...,11.411701,11.169317,13.751442,13.803646,4.892677,4.773806,3.796856,3.577544,4.504385,4.541450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,4,12.658228,8.843270,7.672655,9.125912,5.495477,8.603892,7.747514,5.828978,6.926720,...,13.170441,12.676080,14.124837,13.996436,4.913579,4.399176,3.878855,3.680103,4.726784,4.564637
126,4,12.812823,8.510550,8.729699,9.104402,3.967228,7.719089,7.092496,6.504812,6.157163,...,13.040267,12.403316,13.978009,13.812916,5.189600,4.912618,3.764800,3.664920,4.628355,4.761351
127,4,12.706991,8.795721,7.772359,8.327273,6.329383,8.550471,6.613332,6.308945,7.494852,...,12.825383,12.439265,14.328373,14.008693,4.931460,4.712895,3.913637,3.700964,4.764693,4.834952
128,4,12.684593,8.293938,7.228186,8.494428,6.049414,8.214729,7.287758,5.732710,6.296021,...,13.116581,12.657967,14.390346,14.194904,4.871092,4.739400,3.782980,3.920363,4.665584,4.613326


In [15]:
# Stratify train-val-test split
X_train, X_val, X_test, y_train, y_val, y_test = utils.train_test_split_stratify(df=brain, target='type', SEED=1223)

# Standardization of continuous variables
X_train_scaled, X_val_scaled, X_test_scaled = utils.scale_numerical_variables(X_train, X_test, X_val, numerical_variables = X_train.columns.values.tolist())

Class distribution in train:
1    28
2    20
3    14
4     9
0     7
Name: type, dtype: int64
Class distribution in val:
1    9
2    7
3    4
4    3
0    3
Name: type, dtype: int64
Class distribution in test:
1    9
2    7
3    4
4    3
0    3
Name: type, dtype: int64


In [16]:
X_train_scaled

,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,1405_i_at,1431_at,...,AFFX-r2-Ec-bioD-3_at,AFFX-r2-Ec-bioD-5_at,AFFX-r2-P1-cre-3_at,AFFX-r2-P1-cre-5_at,AFFX-ThrX-3_at,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at
75,0.682064,0.812080,0.345572,0.173856,0.282813,0.684193,0.209364,0.043558,0.580033,0.244462,...,0.667797,0.586658,0.632560,0.485589,0.478970,0.318969,0.157501,0.188098,0.231972,0.374109
118,0.830067,0.303818,0.287039,1.000000,0.308163,0.591153,0.428715,0.484356,0.336788,0.228791,...,0.632048,0.466839,0.573824,0.490832,0.792028,0.634075,0.486110,0.163028,0.311677,0.344999
83,0.260548,0.455918,0.195355,0.786902,0.137949,0.389816,0.338430,0.250551,0.583605,0.219713,...,0.729421,0.619234,0.542287,0.547508,0.598783,0.480383,0.486800,0.240648,0.235842,0.379113
122,0.848069,0.562739,0.230145,0.189416,0.805475,0.857822,0.422318,0.142876,0.162996,0.252044,...,0.721874,0.587323,0.670944,0.606149,0.466806,0.406887,0.550732,0.565437,0.411383,0.363487
92,0.149400,0.586030,0.000000,0.000000,0.119687,0.265315,1.000000,0.231910,0.044782,0.000000,...,0.886642,0.844591,0.912946,0.935962,0.266287,0.166346,0.586971,0.201981,0.478446,0.377520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,0.857154,0.397044,0.240424,0.948545,0.255932,0.886523,0.234805,0.385394,0.263701,0.236133,...,0.563251,0.457280,0.650025,0.581945,0.433880,0.582573,0.337202,0.142356,0.132800,0.501360
87,0.376382,0.535088,0.233811,0.762211,0.250391,0.216820,0.347135,0.554963,0.110032,0.198212,...,0.637219,0.531621,0.484008,0.306322,0.629953,0.355394,0.585183,0.108852,0.290771,0.527451
119,0.680469,0.190325,0.152339,0.678392,0.288332,0.554225,0.126489,0.270648,0.610433,0.178905,...,0.097290,0.108722,0.054618,0.092563,0.481845,0.259945,0.399684,0.146089,0.177436,0.478802
72,0.813770,0.742705,0.310691,0.592037,0.304803,0.442213,0.536939,0.356246,0.450135,0.240659,...,0.648767,0.446650,0.517752,0.512251,0.714882,0.565147,0.366649,0.000000,0.267525,0.422152


* Noselection results

In [17]:
model1 = NoSelectionModel(
    n_inputs=X_train_scaled.columns.values.shape[0],
    n_class=y_train.unique().shape[0]
)       

model1.fit(
    X_train_scaled,
    y_train,
    X_val_scaled,
    y_val,
    epochs=50
)

model1.model.save('results/brain_GSE50161/brain_GSE50161_NoSelectionModel.h5')
with open('results/brain_GSE50161/brain_GSE50161_NoSelectionModel_history.pkl', 'wb') as f:
    pickle.dump(model1.history, f)

Epoch 1/50
3/3 [==============================] - 0s 74ms/step - loss: 80.9297 - sparse_categorical_accuracy: 0.1282 - val_loss: 24.0805 - val_sparse_categorical_accuracy: 0.2692
Epoch 2/50
3/3 [==============================] - 0s 15ms/step - loss: 14.2374 - sparse_categorical_accuracy: 0.2436 - val_loss: 1.6052 - val_sparse_categorical_accuracy: 0.3462
Epoch 3/50
3/3 [==============================] - 0s 13ms/step - loss: 1.8187 - sparse_categorical_accuracy: 0.3462 - val_loss: 1.6038 - val_sparse_categorical_accuracy: 0.3462
Epoch 4/50
3/3 [==============================] - 0s 13ms/step - loss: 1.5993 - sparse_categorical_accuracy: 0.3590 - val_loss: 1.5998 - val_sparse_categorical_accuracy: 0.3462
Epoch 5/50
3/3 [==============================] - 0s 12ms/step - loss: 1.5957 - sparse_categorical_accuracy: 0.3590 - val_loss: 1.5946 - val_sparse_categorical_accuracy: 0.3462
Epoch 6/50
3/3 [==============================] - 0s 12ms/step - loss: 1.5890 - sparse_categorical_accuracy: 0.3

In [18]:
model1.evaluate(X_test_scaled, y_test)

verbose1 = model1.get_verbose()

comparative_results.loc['NoSelection'] = [verbose1['results']['loss'], 
                                          verbose1['results']['accuracy'], 
                                          verbose1['results']['f1'], 
                                          verbose1['selected_features'].shape[0], 
                                          verbose1['selected_features']]
comparative_results

1/1 [==============================] - 0s 18ms/step - loss: 1.5114 - sparse_categorical_accuracy: 0.3462


,test_loss,test_accuracy,test_f1,number of selected features,selected Features
NoSelection,1.511394,0.346154,0.178022,54675,"[1007_s_at, 1053_at, 117_at, 121_at, 1255_g_at..."


* MonoFADL results

In [19]:
model2 = MonoFADLModel(
    n_inputs=X_train_scaled.columns.values.shape[0],
    n_class=y_train.unique().shape[0]
)

model2.fit(
    X_train_scaled,
    y_train,
    X_val_scaled,
    y_val,
    epochs=50
)

model2.model.save('results/brain_GSE50161/brain_GSE50161_MonoFADLModel.h5')
with open('results/brain_GSE50161/brain_GSE50161_MonoFADLModel_history.pkl', 'wb') as f:
    pickle.dump(model2.history, f)

Epoch 1/50
3/3 [==============================] - 1s 88ms/step - loss: 34.1876 - sparse_categorical_accuracy: 0.1923 - val_loss: -9.3222 - val_sparse_categorical_accuracy: 0.3462
Epoch 2/50
3/3 [==============================] - 0s 15ms/step - loss: -13.5057 - sparse_categorical_accuracy: 0.3590 - val_loss: -25.6383 - val_sparse_categorical_accuracy: 0.3462
Epoch 3/50
3/3 [==============================] - 0s 13ms/step - loss: -29.8144 - sparse_categorical_accuracy: 0.3590 - val_loss: -41.8887 - val_sparse_categorical_accuracy: 0.3462
Epoch 4/50
3/3 [==============================] - 0s 14ms/step - loss: -46.0543 - sparse_categorical_accuracy: 0.3590 - val_loss: -58.0312 - val_sparse_categorical_accuracy: 0.3462
Epoch 5/50
3/3 [==============================] - 0s 15ms/step - loss: -62.1126 - sparse_categorical_accuracy: 0.3590 - val_loss: -74.0327 - val_sparse_categorical_accuracy: 0.3462
Epoch 6/50
3/3 [==============================] - 0s 13ms/step - loss: -78.1919 - sparse_categori

In [20]:
model2.evaluate(X_test_scaled, y_test)

verbose2 = model2.get_verbose()

comparative_results.loc['MonoFADL'] = [verbose2['results']['loss'], 
                                          verbose2['results']['accuracy'], 
                                          verbose2['results']['f1'], 
                                          verbose2['selected_features'].shape[0], 
                                          verbose2['selected_features']]
comparative_results

1/1 [==============================] - 0s 15ms/step - loss: -815.5043 - sparse_categorical_accuracy: 0.3462


,test_loss,test_accuracy,test_f1,number of selected features,selected Features
NoSelection,1.511394,0.346154,0.178022,54675,"[1007_s_at, 1053_at, 117_at, 121_at, 1255_g_at..."
MonoFADL,-815.504272,0.346154,0.178022,0,[]


* MultiFADL One-versus-Rest results

In [21]:
model3 = MultiFADLModelOvR(
)

model3.fit(
    X_train_scaled,
    y_train,
    X_val_scaled,
    y_val,
    epochs=50
)

for clas, model in model3.models.items():
    model.model.save(f'results/brain_GSE50161/brain_GSE50161_MultiFADLModel_{clas}.h5')    
    with open(f'results/brain_GSE50161/brain_GSE50161_MultiFADLModel_history_{clas}.pkl', 'wb') as f:
        pickle.dump(model.history, f)

--> Training model class 2 vs rest
Epoch 1/50
3/3 [==============================] - 1s 88ms/step - loss: 38.3934 - binary_accuracy: 0.7436 - val_loss: -10.2328 - val_binary_accuracy: 0.7308
Epoch 2/50
3/3 [==============================] - 0s 14ms/step - loss: -14.4347 - binary_accuracy: 0.7436 - val_loss: -26.6463 - val_binary_accuracy: 0.7308
Epoch 3/50
3/3 [==============================] - 0s 15ms/step - loss: -30.8757 - binary_accuracy: 0.7436 - val_loss: -43.1301 - val_binary_accuracy: 0.7308
Epoch 4/50
3/3 [==============================] - 0s 14ms/step - loss: -47.3616 - binary_accuracy: 0.7436 - val_loss: -59.6497 - val_binary_accuracy: 0.7308
Epoch 5/50
3/3 [==============================] - 0s 15ms/step - loss: -63.9124 - binary_accuracy: 0.7436 - val_loss: -76.2430 - val_binary_accuracy: 0.7308
Epoch 6/50
3/3 [==============================] - 0s 14ms/step - loss: -80.4987 - binary_accuracy: 0.7436 - val_loss: -92.8173 - val_binary_accuracy: 0.7308
Epoch 7/50
3/3 [========

In [22]:
model3.evaluate(X_test_scaled, y_test)

verbose3 = model3.get_verbose()

comparative_results.loc['MultiFADL'] = [verbose3['results']['loss'], 
                                          verbose3['results']['accuracy'], 
                                          verbose3['results']['f1'], 
                                          {clas: verbose3['selected_features'][clas].shape[0] for clas in verbose3['selected_features']}, 
                                          verbose3['selected_features']]

comparative_results

--> Evaluating model class 2 vs rest
1/1 [==============================] - 0s 19ms/step - loss: -794.6967 - binary_accuracy: 0.7308
{'loss': -794.6966552734375, 'accuracy': 0.7307692170143127, 'f1': 0.6170940170940169}
--> Evaluating model class 4 vs rest
1/1 [==============================] - 0s 17ms/step - loss: -816.2164 - binary_accuracy: 0.8846
{'loss': -816.2163696289062, 'accuracy': 0.8846153616905212, 'f1': 0.8304552590266875}
--> Evaluating model class 3 vs rest
1/1 [==============================] - 0s 17ms/step - loss: -801.2966 - binary_accuracy: 0.8462
{'loss': -801.2965698242188, 'accuracy': 0.8461538553237915, 'f1': 0.7756410256410255}
--> Evaluating model class 1 vs rest
1/1 [==============================] - 0s 16ms/step - loss: -792.9927 - binary_accuracy: 0.6538
{'loss': -792.9927368164062, 'accuracy': 0.6538461446762085, 'f1': 0.5169946332737031}
--> Evaluating model class 0 vs rest
1/1 [==============================] - 0s 17ms/step - loss: -843.7061 - binary_accu

,test_loss,test_accuracy,test_f1,number of selected features,selected Features
NoSelection,1.51139,0.346154,0.178022,54675,"[1007_s_at, 1053_at, 117_at, 121_at, 1255_g_at..."
MonoFADL,-815.504,0.346154,0.178022,0,[]
MultiFADL,"{2: -794.6966552734375, 4: -816.2163696289062,...","{2: 0.7307692170143127, 4: 0.8846153616905212,...","{2: 0.6170940170940169, 4: 0.8304552590266875,...","{2: 0, 4: 0, 3: 0, 1: 0, 0: 1, 'global': 1}","{2: [], 4: [], 3: [], 1: [], 0: ['212533_at'],..."


In [23]:
comparative_results.to_csv('results/brain_GSE50161/brain_ComparativeResults.csv')